In [1]:
from collections import Counter, defaultdict
from bs4 import BeautifulSoup as b
from threading import Thread
from tqdm import tqdm
import requests as r
import pandas as pd
import numpy as np
import re

In [2]:
streams = defaultdict()
streams['medical'] = pd.read_csv('../Kunal_mereexams/medical/medical_courses.csv')
streams['eng'] = pd.read_csv('../Kunal_mereexams/engineering/courses_all.csv')
streams['law'] = pd.read_csv('../Kunal_mereexams/law/law_courses.csv')
streams['design'] = pd.read_csv('../Kunal_mereexams/design/design_courses.csv')
streams['it'] = pd.read_csv('../Kunal_mereexams/it_colleges/it_list.csv')
streams['hotel'] = pd.read_csv('../Kunal_mereexams/hotel_manag/hotel_list.csv')
streams['pharmacy'] = pd.read_csv('../Kunal_mereexams/pharmacy/pharmacy_courses.csv')
streams['management'] = pd.read_csv('../Kunal_mereexams/management/mang_list.csv')
streams['media'] = pd.read_csv('../Kunal_mereexams/media_journalism/media_courses.csv')
data = defaultdict()

In [ ]:
for stream in streams.keys():
    data[stream] = pd.DataFrame(columns=['Id', 'Name','known_as', 'Estd','State', 'City', 'Address', 'ownership','Approved_by','Affilate_to','contact', 'Rank','website', 'score', 'facilities', 'exams', 'stream', 'faculty', 'facebook', 'twitter','youtube', 'wiki'], index=range(streams[stream].shape[0]))
    for i in tqdm(range(streams[stream].shape[0])):
        link =  streams[stream].iloc[i,1]
        try:
            res = r.get(link)
        except:
            res = None
            continue
        soup = b(res.content, 'html.parser')

        try:
            also_as = soup.find_all('div',class_='infoQuestion')[0].get_text()[15:]
        except:
            also_as = None

        try:
            name = soup.find_all('h1',class_='titleNameCol')[0].get_text()
        except:
            name = None

        l1 = soup.find_all('ul',class_='clg-info')[0]
        lsts = []
        for li in l1.findAll('li'):
            lsts.append(li.get_text())

        try:
            city, state = lsts[0].split(',')
        except:
            city, state = None,None

        chks = ['Estd : ','Ownership : ','Approved By : ','Affiliated To : ',]
    #     for lst in lsts[1:]:
    #       print(re.sub(chks[0],'',lst))

        estb = None
        ownership = None
        approved_by = None
        affilated_to = None
        chk = False
        for lst in lsts[1:]:
            if chks[0] in lst:
                estb = re.sub(chks[0],'',lst)
                #         print(estb)
                chk = True

    #     print(not chk)
        if (not chk):
            estb = None

        chk = False
        for lst in lsts[1:]:
            if chks[1] in lst:
                ownership = re.sub(chks[1],'',lst)
                #         print(ownership)
                chk = True
        if(not chk):
            ownership = None

        chk = False
        for lst in lsts[1:]:
            if chks[2] in lst:
                approved_by = re.sub(chks[2],'',lst)
                chk = True
        if not chk:
            approved_by = None

        chk = False
        for lst in lsts[1:]:
            if chks[3] in lst:
                affilated_to = re.sub(chks[3],'',lst)
                chk = True
        if not chk:
            affilated_to = None

        try:
            add = soup.find_all('div',class_='mapInfoColInner')[0].find('br').next_sibling
        except:
            add = None      

        l3 = soup.find_all('ul',class_='clg-info')[1].li

        try:
            website = l3 = soup.find_all('ul',class_='clg-info')[1].li.get_text()
        except:
            website = None
        try:
            contact = l3 = soup.find_all('ul',class_='clg-info')[1].li.next_sibling.next_sibling.get_text()[10:]
        except:
            contact = None

        try:
            rank = int(str(soup.find_all('div', class_='countBlockRayco')[0].span)[6:7])
        except:
            rank = None   
        try:
            score = float(soup.find_all('div', class_='countBlockRayco')[1].span.get_text())
        except:
            score = None
        try:
            facilities = []
            facility_list = soup.find('div', class_ = 'facilitylist')
            facility_list.find('ul')
            for li in facility_list.find_all('li'):
                facilities.append(li.get_text())
        except:
            None

        try:
            ex = soup.find_all(class_='college-admission-table')[0].table.tbody
            exams = []
            for tr in ex.find_all('tr'):
                exams.append(tr.td.get_text())
        except:
            None

        try:
            faculty = int(re.findall(r'\d+',str(re.findall(r'Total Faculty In College: \d+',str(soup))))[0])
        except:
            faculty =  None

        social = soup.find_all('div', class_='social-icon-left')[0]
        urls = []
        for li in social.find_all('li'):
            urls.append(li.a['href'])
        face = list(filter(lambda x: 'face' in x, urls))
        twit = list(filter(lambda x: 'twit' in x, urls))
        you = list(filter(lambda x: 'youtube' in x, urls))
        wiki = list(filter(lambda x: 'wiki' in x, urls))
        face, twit, you, wiki = list(map(lambda x: x[0] if len(x) > 0 else None, [face, twit, you, wiki]))

    #     print([i, name, also_as, estb, state, city, add, ownership, approved_by, affilated_to, contact, rank, website, score, facilities, exams, stream])
        data[stream].loc[i] = [i, name, also_as, estb, state, city, add, ownership, approved_by, affilated_to, contact, rank, website, score, facilities, exams, stream, faculty, face, twit, you, wiki]

#     ''':streams[stream].shape[0]]'''

 17%|█▋        | 407/2453 [08:03<40:32,  1.19s/it]

In [7]:
temp = defaultdict()
stream = 'eng'

In [4]:
temp['eng'] = pd.DataFrame(columns = [1,2,3])

In [8]:
temp['eng'] = pd.DataFrame(columns=['Id', 'Name','known_as', 'Estd','State', 'City', 'Address', 'ownership','Approved_by','Affilate_to','contact', 'Rank','website', 'score', 'facilities', 'exams', 'stream'], index=range(streams[stream].shape[0]))

In [9]:
temp['eng']

,Id,Name,known_as,Estd,State,City,Address,ownership,Approved_by,Affilate_to,contact,Rank,website,score,facilities,exams,stream
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
